<a href="https://colab.research.google.com/github/jamshedulhaquesheikh00/Jamshed_Ul_Haque_Sheikh_April_codings_nights_0001/blob/main/05_secure_data_encryption_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# secure_data_encryption_assignment.md

# 🔹 Starter Code (Python + Streamlit)

In [ ]:
import streamlit as st
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding as rsa_padding
from cryptography.hazmat.primitives import serialization
import os
import base64  # For encoding/decoding keys for display/input

# --- Helper Functions ---
def display_key_warning():
    st.warning("**Advertencia de seguridad importante:** Las claves generadas se muestran solo con fines de demostración. **No almacene ni comparta estas claves de forma insegura en una aplicación del mundo real.** La gestión segura de claves es crucial para la seguridad de sus datos cifrados.")

# --- Task 1: Symmetric Encryption (AES) ---

def generar_clave_aes():
    """Genera una nueva clave AES."""
    clave = os.urandom(32)  # Clave de 256 bits
    return base64.b64encode(clave)

def encryptar_aes(texto_plano, clave_bytes_base64):
    """Cifra el texto plano utilizando AES-CBC con un IV aleatorio."""
    clave_bytes = base64.b64decode(clave_bytes_base64)
    iv = os.urandom(16)
    cifrador = Cipher(algorithms.AES(clave_bytes), modes.CBC(iv))
    encriptor = cifrador.encryptor()
    rellenador = padding.PKCS7(algorithms.AES.block_size).padder()
    texto_plano_rellenado = rellenador.update(texto_plano.encode('utf-8')) + rellenador.finalize()
    texto_cifrado = encriptor.update(texto_plano_rellenado) + encriptor.finalize()
    return base64.b64encode(iv + texto_cifrado)

def decryptar_aes(texto_cifrado_con_iv_base64, clave_bytes_base64):
    """Descifra el texto cifrado utilizando AES-CBC."""
    texto_cifrado_con_iv = base64.b64decode(texto_cifrado_con_iv_base64)
    clave_bytes = base64.b64decode(clave_bytes_base64)
    iv = texto_cifrado_con_iv[:16]
    texto_cifrado = texto_cifrado_con_iv[16:]
    cifrador = Cipher(algorithms.AES(clave_bytes), modes.CBC(iv))
    desencriptor = cifrador.decryptor()
    desrellenador = padding.PKCS7(algorithms.AES.block_size).unpadder()
    texto_plano_rellenado_descifrado = desencriptor.update(texto_cifrado) + desencriptor.finalize()
    texto_plano = desrellenador.update(texto_plano_rellenado_descifrado) + desrellenador.finalize()
    return texto_plano.decode('utf-8')

# --- Task 2: Asymmetric Encryption (RSA) ---

def generar_par_claves_rsa():
    """Genera un nuevo par de claves pública/privada RSA."""
    clave_privada = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    clave_publica = clave_privada.public_key()
    return clave_publica, clave_privada

def encryptar_rsa(texto_plano, clave_publica_pem):
    """Cifra el texto plano utilizando la clave pública RSA."""
    clave_publica = serialization.load_pem_public_key(
        clave_publica_pem.encode('utf-8')
    )
    texto_cifrado = clave_publica.encrypt(
        texto_plano.encode('utf-8'),
        rsa_padding.OAEP(
            mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return base64.b64encode(texto_cifrado)

def decryptar_rsa(texto_cifrado_base64, clave_privada_pem):
    """Descifra el texto cifrado utilizando la clave privada RSA."""
    texto_cifrado = base64.b64decode(texto_cifrado_base64)
    clave_privada = serialization.load_pem_private_key(
        clave_privada_pem.encode('utf-8'),
        password=None  # Sin contraseña para simplificar en esta tarea
    )
    texto_plano = clave_privada.decrypt(
        texto_cifrado,
        rsa_padding.OAEP(
            mgf=rsa_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return texto_plano.decode('utf-8')

def serializar_clave_publica(clave_publica):
    """Serializa una clave pública RSA al formato PEM."""
    clave_publica_pem = clave_publica.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    return clave_publica_pem.decode('utf-8')

def serializar_clave_privada(clave_privada):
    """Serializa una clave privada RSA al formato PEM."""
    clave_privada_pem = clave_privada.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    return clave_privada_pem.decode('utf-8')

# --- Streamlit Application ---

st.title("Cifrado Seguro de Datos")
display_key_warning()

# --- Sección AES ---
st.header("Cifrado/Descifrado AES")

aes_texto_plano = st.text_area("Texto Plano (AES):")
generar_clave_aes_boton = st.button("Generar Clave AES")
if generar_clave_aes_boton:
    clave_aes = generar_clave_aes()
    st.session_state['clave_aes'] = clave_aes.decode('utf-8')
    st.info(f"Clave AES Generada (Base64): {st.session_state['clave_aes']}")

clave_aes_proporcionada_str = st.text_input("Clave AES (codificada en Base64):")
if clave_aes_proporcionada_str:
    try:
        base64.b64decode(clave_aes_proporcionada_str)
        st.session_state['clave_aes'] = clave_aes_proporcionada_str
    except Exception as e:
        st.error(f"Formato de clave AES inválido: {e}")

if 'clave_aes' in st.session_state:
    if st.button("Cifrar (AES)"):
        if aes_texto_plano:
            texto_cifrado_aes = encryptar_aes(aes_texto_plano, st.session_state['clave_aes'])
            st.success(f"Texto Cifrado (AES - Base64): {texto_cifrado_aes.decode('utf-8')}")
        else:
            st.warning("Por favor, introduzca texto plano para cifrar.")

aes_texto_cifrado_base64 = st.text_input("Texto Cifrado (AES - Base64):")
clave_aes_descifrado_str = st.text_input("Clave AES para Descifrado (codificada en Base64):")

if st.button("Descifrar (AES)"):
    if aes_texto_cifrado_base64 and clave_aes_descifrado_str:
        try:
            texto_plano_descifrado_aes = decryptar_aes(aes_texto_cifrado_base64, clave_aes_descifrado_str)
            st.info(f"Texto Plano Descifrado (AES): {texto_plano_descifrado_aes}")
        except Exception as e:
            st.error(f"Error de descifrado: {e}")
    else:
        st.warning("Por favor, introduzca texto cifrado y la clave de descifrado.")

# --- Sección RSA ---
st.header("Cifrado/Descifrado RSA")

generar_claves_rsa_boton = st.button("Generar Par de Claves RSA")
if generar_claves_rsa_boton:
    clave_publica_rsa, clave_privada_rsa = generar_par_claves_rsa()
    clave_publica_pem = serializar_clave_publica(clave_publica_rsa)
    clave_privada_pem = serializar_clave_privada(clave_privada_rsa)
    st.session_state['clave_publica_rsa'] = clave_publica_pem
    st.session_state['clave_privada_rsa'] = clave_privada_pem
    st.info(f"Clave Pública RSA Generada (formato PEM):\n```\n{clave_publica_pem}\n```")
    st.warning("**Advertencia de seguridad importante:** La clave privada generada se muestra solo con fines de demostración. **No almacene ni comparta esta clave privada de forma insegura en una aplicación del mundo real.** La gestión segura de claves es crucial para la seguridad de sus datos cifrados.")
    st.text_area("Clave Privada RSA Generada (formato PEM - **MANTENER EN SECRETO!**):", clave_privada_pem, height=200)

rsa_texto_plano_cifrado = st.text_area("Texto Plano (RSA - Cifrado):")
clave_publica_rsa_proporcionada_pem = st.text_area("Clave Pública RSA (formato PEM) para Cifrado:")

if st.button("Cifrar (RSA)"):
    if rsa_texto_plano_cifrado and clave_publica_rsa_proporcionada_pem:
        try:
            texto_cifrado_rsa = encryptar_rsa(rsa_texto_plano_cifrado, clave_publica_rsa_proporcionada_pem)
            st.success(f"Texto Cifrado (RSA - Base64): {texto_cifrado_rsa.decode('utf-8')}")
        except Exception as e:
            st.error(f"Error de cifrado RSA: {e}")
    else:
        st.warning("Por favor, introduzca texto plano y la clave pública RSA para el cifrado.")

rsa_texto_cifrado_base64_descifrado = st.text_input("Texto Cifrado (RSA - Base64) para Descifrado:")
clave_privada_rsa_proporcionada_pem = st.text_area("Clave Privada RSA (formato PEM) para Descifrado:", height=200)

if st.button("Descifrar (RSA)"):
    if rsa_texto_cifrado_base64_descifrado and clave_privada_rsa_proporcionada_pem:
        try:
            texto_plano_descifrado_rsa = decryptar_rsa(rsa_texto_cifrado_base64_descifrado, clave_privada_rsa_proporcionada_pem)
            st.info(f"Texto Plano Descifrado (RSA): {texto_plano_descifrado_rsa}")
        except Exception as e:
            st.error(f"Error de descifrado RSA: {e}")
    else:
        st.warning("Por favor, introduzca texto cifrado y la clave privada RSA para el descifrado.")

# --- Consideraciones sobre la gestión de claves ---
st.header("Gestión de Claves")
st.markdown("""
**Importancia de la Gestión Segura de Claves:**

La gestión segura de claves es absolutamente crítica para la efectividad de cualquier sistema de cifrado. Si las claves de cifrado se ven comprometidas, la seguridad de los datos cifrados se pierde por completo, independientemente de la solidez del algoritmo de cifrado en sí. El almacenamiento o la transmisión inseguros de las claves pueden provocar accesos no autorizados, filtraciones de datos y graves vulnerabilidades de seguridad.

**Riesgos del Almacenamiento y la Transmisión Inseguros de Claves:**

* **Exposición al Acceso No Autorizado:** Almacenar claves en ubicaciones de fácil acceso (por ejemplo, archivos de texto plano, codificados directamente en las aplicaciones) las hace vulnerables al robo o la divulgación accidental.
* **Ataques de Intermediario (Man-in-the-Middle):** La transmisión de claves a través de canales inseguros (por ejemplo, conexiones de red no cifradas) permite a los atacantes interceptarlas.
* **Amenazas Internas:** Las malas prácticas de gestión de claves pueden aumentar el riesgo de que personas con malas intenciones dentro de la organización obtengan acceso a claves sensibles.
* **Pérdida de Datos:** Si las claves se pierden o se eliminan accidentalmente sin copias de seguridad adecuadas, los datos cifrados se vuelven permanentemente inaccesibles.
* **Incumplimiento de Normativas:** Muchas regulaciones y estándares exigen que las organizaciones implementen prácticas sólidas de gestión de claves. El incumplimiento puede resultar en sanciones.

**Métodos Prácticos para el Almacenamiento y la Gestión Segura de Claves:**

1.  **Módulos de Seguridad de Hardware (HSM):** Los HSM son dispositivos de hardware dedicados diseñados para almacenar y gestionar de forma segura las claves criptográficas. Proporcionan un entorno resistente a la manipulación y, a menudo, imponen políticas estrictas de control de acceso. Las claves almacenadas en los HSM generalmente nunca se exponen en texto plano fuera del dispositivo.

2.  **Sistemas de Gestión de Claves (KMS):** Los KMS son soluciones basadas en software que proporcionan una gestión centralizada de las claves de cifrado en toda una organización. Ofrecen funciones como la generación, el almacenamiento, la distribución, la rotación, el control de acceso, la auditoría y la gestión del ciclo de vida de las claves. Los KMS pueden integrarse con diversas aplicaciones y servicios.

3.  **Herramientas de Gestión de Secretos (por ejemplo, HashiCorp Vault, CyberArk):** Estas herramientas están diseñadas para almacenar y gestionar de forma segura secretos, incluidas las claves de cifrado, los tokens de API y las contraseñas. A menudo proporcionan funciones como el control de acceso, el registro de auditoría y la rotación de secretos.

4.  **Servicios de Gestión de Claves Basados en la Nube (por ejemplo, AWS KMS, Azure Key Vault, Google Cloud KMS):** Los proveedores de la nube ofrecen servicios de gestión de claves gestionados que simplifican el proceso de creación, almacenamiento y control de las claves de cifrado utilizadas para proteger los datos en sus entornos de nube. Estos servicios a menudo se integran con otros servicios de la nube y ofrecen funciones como la disponibilidad regional y la rotación automática de claves.

5.  **Enclaves Seguros (por ejemplo, Intel SGX, ARM TrustZone):** Estas son regiones aisladas y seguras dentro de un procesador que pueden proteger datos y código sensibles, incluidas las claves de cifrado, del resto del sistema, incluso si el sistema operativo o el hipervisor están comprometidos.

Es crucial elegir soluciones y prácticas de gestión de claves que se alineen con los requisitos de seguridad específicos y el perfil de riesgo de la aplicación y la sensibilidad de los datos que se protegen.
""")